In [ ]:
# import netCDF4 as nc

# import re
# import itertools
# import time

# import datetime as dt
# from urllib.request import urlopen

# from ast import literal_eval
# from datetime import timedelta


# import urllib3

In [ ]:
## REMOVE
def deploy_depth_check(refdes):
    # 
    # deployment locations
    deploy_depth = {}
    dr_data = refdes_datareview_json(refdes)
    for i, d in enumerate(dr_data['instrument']['deployments']):
        deploy_depth[i] = {}
        deploy_depth[i]['deployment'] = d['deployment_number']
        deploy_depth[i]['d_depth'] = d['deployment_depth']
        deploy_depth[i]['water_depth'] = d['water_depth']
        

    # put info in a data frame
    df_depth = pd.DataFrame.from_dict(deploy_depth, orient='index').sort_index()

    return df_depth
## REMOVE
df_depth = deploy_depth_check('GP03FLMB-RIM01-02-CTDMOG060')
df_depth

In [ ]:
## REMOVE
def sensor_annotation_check(refdes):
    # deployment start and end dates 
    sensor_anno = {}
    dr_data = refdes_datareview_json(refdes)
    for i, d in enumerate(dr_data['instrument']['annotations']):
        sensor_anno[i] = {}
        sensor_anno[i]['reference_designator'] = d['reference_designator']
        sensor_anno[i]['annotation'] = d['annotation']
        sensor_anno[i]['start_datetime'] = d['start_datetime']
        sensor_anno[i]['end_datetime'] = d['end_datetime']

    # put info in a data frame
    df_anno = pd.DataFrame.from_dict(sensor_anno, orient='index').sort_index()
    
    return df_anno
## REMOVE
df_anno = sensor_annotation_check('GP03FLMB-RIM01-02-CTDMOG060')
pd.set_option('display.max_colwidth', None)
df_anno

In [ ]:
## REMOVE
def deploy_location_check(refdes):
    # Calculate the distance in kilometers between an instrument's location (defined in asset management) and previous
    # deployment locations
    deploy_loc = {}
    dr_data = refdes_datareview_json(refdes)
   
    for i, d in enumerate(dr_data['instrument']['deployments']):
        deploy_loc[i] = {}
        deploy_loc[i]['deployment'] = d['deployment_number']
        deploy_loc[i]['lat'] = d['latitude']
        deploy_loc[i]['lon'] = d['longitude']

    # put info in a data frame
    df = pd.DataFrame.from_dict(deploy_loc, orient='index').sort_index()
    
    y = {}
    
    for i, k in df.iterrows():
        if i > 0:
            loc1 = [k['lat'], k['lon']]
            d1 = int(k['deployment'])
            for x in range(i):
                info0 = df.iloc[x]
                compare = 'diff_km_D{}_to_D{}'.format(d1, int(info0['deployment']))
                loc0 = [info0['lat'], info0['lon']]
                diff_loc = round(geodesic(loc0, loc1).kilometers, 4)
                y.update({compare: diff_loc})
    return df, y
## REMOVE
location, distance = deploy_location_check('GP03FLMB-RIM01-02-CTDMOG060')
## REMOVE
location

In [ ]:
from datetime import datetime

now = datetime.now()

current_time = now.strftime("%Y-%m-%dT%H:%M:%S")
print("Current Time =", current_time)
if dates[-1] is None:
    dates[-1] = current_time
dates

In [ ]:
ind  = sorted(range(len(dates)),key=dates.__getitem__)
date_list = [dates[i].split('+')[0] for i in ind ]
deployment_list = [names[i] for i in ind ]
date_list, deployment_list

In [ ]:
# Make a histogram
for i in range(len(list_recovered)):
    d1 = xr.open_dataset(list_recovered['files'][i])
    d1 = d1.swap_dims({'obs': 'time'})
    d1['ctdmo_seawater_temperature'].plot.hist(bins=100);

In [ ]:
 dinfo = {}
    if len(rdm_filtered) == 1:
        print('Only one delivery method provided - no comparison.')
        continue
    elif len(rdm_filtered) > 1 & len(rdm_filtered) <= 3:
        print('\nComparing data from different methods for: {}'.format(r))
        for i in range(len(rdm_filtered)):
            urls = [x for x in url_list if rdm_filtered[i] in x]
            for u in urls:
                splitter = u.split('/')[-2].split('-')
                catalog_rms = '-'.join((r, splitter[-2], splitter[-1]))
                udatasets = cf.get_nc_urls([u])
                deployments = [str(k.split('/')[-1][0:14]) for k in udatasets]
                udeploy = np.unique(deployments).tolist()

In [ ]:
rd_list = []
rdm_list = []
for uu in url_list:
    elements = uu.split('_')[1].split('-')
    rd = '-'.join((elements[0], elements[1], elements[2], elements[3]))
    rdm = uu.split('-')[4]
   
 
    if rd not in rd_list:
        rd_list.append(rd)
    if rdm not in rdm_list:
        rdm_list.append(rdm)
print(rd_list, rdm_list)

for r in rd_list:
    rdm_filtered = [k for k in rdm_list if r in k]
    print(r, rdm_filtered)

In [ ]:
 dinfo = {}
    if len(rdm_filtered) == 1:
        print('Only one delivery method provided - no comparison.')
        continue
    elif len(rdm_filtered) > 1 & len(rdm_filtered) <= 3:
        print('\nComparing data from different methods for: {}'.format(r))
        for i in range(len(rdm_filtered)):
            urls = [x for x in url_list if rdm_filtered[i] in x]
            for u in urls:
                splitter = u.split('/')[-2].split('-')
                catalog_rms = '-'.join((r, splitter[-2], splitter[-1]))
                udatasets = cf.get_nc_urls([u])
                deployments = [str(k.split('/')[-1][0:14]) for k in udatasets]
                udeploy = np.unique(deployments).tolist()

Data Coordinates Function.

In [ ]:
def missing_coordinates(datasets):
    """
    Check file coordinates agains the expected coordinates array ['obs', 'time', 'pressure', 'lat', 'lon']
    """
    file_coordinates = [x for x in tuple(datasets.coords.keys())]

    missing_coords = [x for x in ['obs', 'time', 'pressure', 'lat', 'lon'] if x not in file_coordinates]
    return missing_coords


In [ ]:
# check variable coordinates
missing_coords = missing_coordinates(ds)

In [ ]:
df0 = pd.DataFrame({'sv': [sv], 'var_units': [units], 
                    'Missing Coordinates':[(coords, 'missing:',missing_coords)]  
                   }, index = [deployment]) 
df = df.append(df0)

pd.set_option('display.max_colwidth', -1)
df

The missing coordinates in the parameter attributes should be addressed to fix the data integrity and usability with other datasets.